In order to create our own module in Elixir, we use the `defmodule` macro. We use the `def` macro to define functions in that module

In [1]:
defmodule Math do
  def sum(a, b) do
    a + b
    end
    end

{:module, Math, <<70, 79, 82, 49, 0, 0, 4, 20, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 124, 0, 0, 0, 14, 11, 69, 108, 105, 120, 105, 114, 46, 77, 97, 116, 104, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, ...>>, {:sum, 2}}

In [5]:
Math.sum(1, 2)

3

## Compilation

Most of the time it is convenient to write modules into files so they can be compiled and reused. Let's assume we have a file named `math.ex` with the following contents:

In [6]:
defmodule Math do
  def sum(a, b) do
    a + b
  end
end


{:module, Math, <<70, 79, 82, 49, 0, 0, 4, 20, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 124, 0, 0, 0, 14, 11, 69, 108, 105, 120, 105, 114, 46, 77, 97, 116, 104, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, ...>>, {:sum, 2}}

This file can be compiled using `elixirc`

In [7]:
elixirc math.ex

CompileError: 1

This will generate a file named `Elixir.Math.beam` containing the bytecode for the defined module. If we start `iex` again, our module definition will be available(provided that `iex` is started in the same directory the bytecode file is in)

Elixir projects are usually organized into three directies:

1. ebin - contains the compiled bytecode
2. lib - contains elixir code (usually `.ex` files)
3. test - contains tests(usually `.exs` files)

In [3]:
defmodule Math do
  def sum(a, b) do
    a + b
  end
end

IO.puts Math.sum(1, 2)

3


:ok

In [4]:
elixir math.exs

CompileError: 1

The file will be compiled in memory and executed, printing "3" as the result. No bytecode file will be created. 

In [7]:
defmodule Math do
def sum(a, b) do
do_sum(a, b)
end

defp do_sum(a, b) do
a+b
end
end



{:module, Math, <<70, 79, 82, 49, 0, 0, 4, 96, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 131, 0, 0, 0, 15, 11, 69, 108, 105, 120, 105, 114, 46, 77, 97, 116, 104, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, ...>>, {:do_sum, 2}}

In [8]:
IO.puts Math.sum(1, 2)

3


:ok

In [9]:
IO.puts Math.do_sum(1, 2)

UndefinedFunctionError: 1

Function declarations also support guards and multiple clauses. If a function has several clauses, Elixir will try each clause until it finds one that matches. Here is an implementation of a function that checks if the given number is zero or not

In [9]:
defmodule Math do
  def zero?(0) do
    true
    end
    
  def zero?(x) when is_integer(x) do
    false
    end
    end

{:module, Math, <<70, 79, 82, 49, 0, 0, 4, 52, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 135, 0, 0, 0, 15, 11, 69, 108, 105, 120, 105, 114, 46, 77, 97, 116, 104, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, ...>>, {:zero?, 1}}

In [10]:
Math.zero?(0)

true

In [11]:
Math.zero?(1)

false

In [12]:
Math.zero?([1,2,3])

FunctionClauseError: 1

In [12]:
Math.zero?(0.0)

FunctionClauseError: 1

Giving an argument that does not match any of the clauses raises an error.

Similar to constructs like `if`, named functions support `do:` and `do/end` block syntax. For example, we can edit `math.exs` to look like this:

In [12]:
defmodule Math do
  def zero?(0), do: true
  def zero?(x)  when is_integer(x), do: false
  end

{:module, Math, <<70, 79, 82, 49, 0, 0, 4, 52, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 135, 0, 0, 0, 15, 11, 69, 108, 105, 120, 105, 114, 46, 77, 97, 116, 104, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, ...>>, {:zero?, 1}}

In [13]:
fun = &Math.zero?/1

&Math.zero?/1

In [14]:
is_function(fun)

true

In [15]:
fun.(0)

true

The capture operator bridges this gap by allowing named functions to be assigned to variables and passed as arguments in the same way we assign, invoke and pass anonymous functions.

In [16]:
&is_function/1

&:erlang.is_function/1

In [17]:
(&is_function/1).(fun)

true

Note the captrue syntax can also be used as a shortcut for creating functions

In [18]:
fun = &(&1 + 1)

#Function<6.128620087/1 in :erl_eval.expr/5>

In [19]:
fun.(1)

2

In [20]:
fun2 = &"Good #{&1}"

#Function<6.128620087/1 in :erl_eval.expr/5>

In [21]:
fun2.("morning")

"Good morning"

The `&1` represents the first argument passed into function. `&(&1 + 1)` above is exactly the same as `fn x -> x + 1 end`. The syntax above is useful for short function definitions.

If you want to capture a function from a module, you can do `&Module.function()`

In [22]:
fun = &List.flatten(&1, &2)

&List.flatten/2

In [23]:
fun.([1, [[2], 3]], [4,5])

[1, 2, 3, 4, 5]

In [26]:
List.flatten([1, [[2], 3]], [4,5])

[1, 2, 3, 4, 5]

`&List.flatten(&1, &2)` is the same as writing `fn(list, tail) -> List.flatten(list, tail) end` which in this case is equivalent to `&List.flatten/2`   

## Default arguments

In [27]:
defmodule Concat do
  def join(a, b, sep \\ " ") do
    a <> sep <> b
    end
    end

{:module, Concat, <<70, 79, 82, 49, 0, 0, 4, 240, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 138, 0, 0, 0, 15, 13, 69, 108, 105, 120, 105, 114, 46, 67, 111, 110, 99, 97, 116, 8, 95, 95, 105, 110, 102, 111, 95, 95, ...>>, {:join, 3}}

In [28]:
Concat.join("hello", "world")

"hello world"

In [29]:
Concat.join("Hello", "world", "_")

"Hello_world"

In [30]:
defmodule Concat do
  def join(a, b \\ nil, sep \\ " ")
  def join(a, b, _sep) when is_nil(b) do
  a
  end
  
  def join(a, b, sep) do
  a <> sep <> b
  end
  end

{:module, Concat, <<70, 79, 82, 49, 0, 0, 5, 132, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 142, 0, 0, 0, 16, 13, 69, 108, 105, 120, 105, 114, 46, 67, 111, 110, 99, 97, 116, 8, 95, 95, 105, 110, 102, 111, 95, 95, ...>>, {:join, 3}}

In [31]:
Concat.join("Hello", "world")

"Hello world"

In [32]:
Concat.join("Hello", "world", "_")

"Hello_world"

In [33]:
Concat.join("Hello")

"Hello"

In [35]:
defmodule Concat do
def join(a, b) do
IO.puts "***First join"
a <> b
end

def join(a, b, sep \\ " ") do
IO.puts "***Second join"
a <> sep <> b
end
end

{:module, Concat, <<70, 79, 82, 49, 0, 0, 5, 216, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 153, 0, 0, 0, 17, 13, 69, 108, 105, 120, 105, 114, 46, 67, 111, 110, 99, 97, 116, 8, 95, 95, 105, 110, 102, 111, 95, 95, ...>>, {:join, 3}}

If we save the code above in a file named "concat.ex" and compile it, Elixir will emit the following warning:

> warning: this clause match because a previous clause at line 2 always matches

In [37]:
Concat.join "hello", "world"

***First join


"helloworld"

In [38]:
Concat.join "Hello", "world", "_"

***Second join


"Hello_world"